**Marco de la investigacion**

Ataques basados en betweenness, recalculados e iniciales, con y sin cutoff sobre redes de Delaunay, con y sin condiciones de borde periodicas

**Interrogantes relevantes**

Orden de las transiciones y exponentes criticos

Valor de los puntos criticos




In [6]:
import os
import sys
import numpy as np
import pandas as pd
import igraph as ig
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../python')

from mpl_settings_v3 import *
from plot_auxiliary import *
from planar import create_proximity_graph
from auxiliary import get_base_network_name
from auxiliary import getLinearReg
from plot_auxiliary import load_dataframe, get_critical_measures

%matplotlib inline

net_type = 'DT'
param = 'param'

net_dir = os.path.join('..', 'networks', net_type)

## Scaling de S y meanS - Ataque Btw

In [7]:
def load_dataframes(net_type, param, attack, sizes, min_seeds=100):
    dfs = {}
    for N in sizes:
        try:
            dfs[N] = load_dataframe(net_type, N, param, attack, min_nseeds=min_seeds)
        except FileNotFoundError:
            continue
    return dfs

def set_colormap(n_colors):
    return sns.cubehelix_palette(
        n_colors, start=2, rot=0.3, light=0.8, dark=0.1
    )

class ExponentCalculator:
    def __init__(
        self,
        net_type,
        param,
        attack,
        sizes,
        min_seeds=100,
        metrics=['meanS', 'Nsec'],
        pc=0
    ):
        self.dfs = load_dataframes(net_type, param, attack, sizes, min_seeds=min_seeds)
        self.pc = pc
        self.metrics = metrics
        self.exponents = {}
        self.L_values = np.sqrt(sizes)
        
    def run(self):
        X = self.L_values
        for i, metric in enumerate(self.metrics):
            if metric not in ['Nsec', 'meanS']:
                continue
            metric_values = get_critical_measures(self.dfs, metric, fc='peak')
            Y_pred, slope, y_err = getLinearReg(X, metric_values)
            if metric == 'Nsec':
                self.exponents['beta_over_nu'] = 2 - slope
                delta_beta = y_err
            elif metric == 'meanS':
                self.exponents['gamma_over_nu'] = slope
                delta_gamma = y_err

                Y, _ = getPeaks(self.dfs, metric)
                Y_pred, slope, y_err = getLinearReg(X, Y)
                self.exponents['one_over_nu'] = -slope
        return

class CollapsePlotter:
    
    def __init__(
        self, 
        net_type, 
        param, 
        attack,
        sizes,
        min_seeds, 
        metrics=['Sgcc', 'meanS'], 
        exponents=None
    ):
        if exponents:
            self.exponents = exponents
        else:
            expCalculator = ExponentCalculator(
                net_type, 
                param, 
                attack,
                sizes,
                min_seeds, 
                metrics=metrics, 
            )
            expCalculator.run()
            self.exponents = expCalculator.exponents
        self.dfs = load_dataframes(net_type, param, attack, sizes, min_seeds=min_seeds)
        self.metrics = metrics
        self.colors = set_colormap(len(sizes))

    def plot_metric(self, ax, metric, scaled=False):
        for i, (N, df) in enumerate(self.dfs.items()):
            L = np.sqrt(N)
            X = df.f
            Y = df[metric]
            if scaled:
                X = X*L**(self.exponents['one_over_nu'])
                ax.set_xlabel(r'$f L^{1/\nu}$', fontsize=26)
                if metric == 'Sgcc':
                    Y = Y*L**(self.exponents['beta_over_nu'])
                    ax.set_ylabel(r'$S_1 L^{\beta/\nu}$', fontsize=26)
                elif metric == 'meanS':
                    Y = Y*L**(-self.exponents['gamma_over_nu'])
                    ax.set_ylabel(r'$\langle s\rangle L^{-\gamma/\nu}$', fontsize=26)
                    ax.set_xlabel(r'$f L^{1/\nu}$', fontsize=26)
                elif metric == 'Nsec':
                    Y = Y*L**(self.exponents['beta_over_nu']-2)
                    ax.set_ylabel(r'$N_2 L^{\beta/\nu-2}$', fontsize=26)
            else:
                ax.set_xlabel(r'$f$')
                if metric == 'Sgcc':
                    ax.set_ylabel(r'$S_1$')
                elif metric == 'meanS':
                    ax.set_ylabel(r'$\langle s\rangle$')
                elif metric == 'Nsec':
                    ax.set_ylabel(r'$N_2$')      
            ax.plot(X, Y, color=self.colors[i])
        
    def plot(self, how=['non_scaled']):
        
        ncols = len(self.metrics)
        nrows = len(how)
        fig, axes = plt.subplots(
            figsize=(8*ncols, 6*nrows), ncols=ncols, nrows=nrows
        )
        fig.suptitle(attack_dict[attack])
        if len(axes.shape) == 1:
            axes = [axes]
        for j, option in enumerate(how):
            scaled = True if option == 'scaled' else False
            for i, ax in enumerate(axes[j]):
                self.plot_metric(ax, self.metrics[i], scaled=scaled)
            plt.tight_layout()
        
        return fig, axes


def get_exponents(attack):    
    if attack == 'BtwU':
        one_over_nu = 0.57
        gamma_over_nu = 2.05
        beta_over_nu = 0
    elif attack == 'Btw':
        one_over_nu = 0.57
        gamma_over_nu = 2.05
        beta_over_nu = 0

    nu = 1 / one_over_nu
    gamma = gamma_over_nu * nu
    beta = beta_over_nu * nu

    exponents = {
        'nu': nu,
        'gamma': gamma,
        'beta': beta
    }
    return exponents

In [8]:
from typing import Dict, Iterable

class AttackStats:
    
    def __init__(
        self,
        sizes: Iterable[int],
        exponents: Dict[str, float],
        pc: float,
    ) -> None:
        
        self.sizes = sizes
        self.exponents = exponents
        self.pc = pc

In [9]:
attack = 'BtwU_cutoff3'

if attack == 'Btw':
    sizes = [2**i for i in range(12, 17)]
elif attack == 'BtwU':
    sizes = [1024, 2048, 4096, 8192]#, 16384]
else:
    sizes = [1024, 2048, 4096, 8192, 16384]
print(sizes)

min_seeds = 100

plotter = CollapsePlotter(
    net_type, 
    param,
    attack,
    sizes,
    min_seeds, 
    metrics=['Sgcc', 'meanS', 'Nsec']
)
print(plotter.exponents)
fig, axes = plotter.plot(how=['non_scaled', 'scaled'])

for ax in axes[0]:
    ax.set_xlim(0, 0.25)
    
for ax in axes[1]:
    ax.set_xlim(0, 1)

[1024, 2048, 4096, 8192, 16384]


TypeError: expected x and y to have same length

In [ ]:
class PeakPlotter:
    
    def __init__(
        self, 
        net_type, 
        param, 
        attack,
        sizes,
        min_seeds, 
        metrics=['Sgcc', 'meanS'], 
        exponents=None
    ):
        self.exponents = exponents
        self.dfs = load_dataframes(net_type, param, attack, min_seeds=min_seeds)
        self.metrics = metrics  

In [ ]:
exponentCalculator = ExponentCalculator(
    net_type, 
    param,
    attack,
    sizes,
    min_seeds 
)
exponentCalculator.run()
exponentCalculator.exponents